In [ ]:
# 文件列表
files = [
    "D:/Permafrost/mask_re/mean/pfr_mean_1_97_01.nc",
    "D:/Permafrost/mask_re/mean/pfr_mean_2_02_06.nc",
    "D:/Permafrost/mask_re/mean/pfr_mean_3_07_11.nc",
    "D:/Permafrost/mask_re/mean/pfr_mean_4_12_16.nc",
    "D:/Permafrost/mask_re/mean/pfr_mean_5_17_21.nc",
]
folder_path = 'D:/Permafrost/mask_re/mean'

In [ ]:
# 冻土区面积空间演播图
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
import matplotlib.ticker as mticker
from matplotlib.colors import ListedColormap, BoundaryNorm

# 设置中文字体（可选，如果标题需要中文）
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False

# 设置文件夹路径
folder_path = 'D:/Permafrost/mask_re/mean'  # 修改为您的文件夹路径
nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]

# 为每个文件创建单独的图形
for i, file in enumerate(nc_files):
    print(f"正在处理文件: {file}")
    
    # 创建图形和投影
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0, central_latitude=90))
    
    # 读取数据
    ds = xr.open_dataset(os.path.join(folder_path, file))
    pfr_data = ds['PFR_mean']
    #
    colors = ["#2469A2", "#0A3C64"]
    cmap = ListedColormap(colors)
    bounds = [0, 90, 100]
    norm = BoundaryNorm(bounds, cmap.N)
    # 绘制填色图
    im = pfr_data.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(),
                                 cmap=cmap, add_colorbar=False,
                                 robust=True)  # robust=True可以排除异常值的影响
    
    # 添加地图特征
    ax.add_feature(cfeature.COASTLINE, linewidth=0.8, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray')
    ax.add_feature(cfeature.OCEAN, color='lightblue', alpha=0.3)
    ax.add_feature(cfeature.LAND, color='lightgray', alpha=0.3)
    
    # 设置网格线 - 经度间隔30度，纬度间隔20度
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.7, color='gray')
    
    # 设置经度标签 - 每30度一个
    gl.xlocator = mticker.FixedLocator(np.arange(-180, 181, 30))
    # 设置纬度标签 - 每20度一个（北半球0-90度）
    gl.ylocator = mticker.FixedLocator(np.arange(0, 91, 20))
    
    # 控制标签显示位置
    gl.top_labels = True    # 不显示顶部标签
    gl.right_labels = True  # 不显示右侧标签
    gl.left_labels = True    # 显示左侧标签
    gl.bottom_labels = True  # 显示底部标签
    
    # 设置标签样式
    gl.xlabel_style = {'size': 25, 'color': 'black'}
    gl.ylabel_style = {'size': 25, 'color': 'black'}
    
    # 设置北半球范围
    ax.set_extent([-180, 180, 25, 90], crs=ccrs.PlateCarree())
    
    plt.tight_layout()
    
    # 保存图片（可选）
    plt.savefig(f'PFR_mean_plot_{i+1}.png', dpi=300)
    
    plt.show()
    # 关闭数据集
    ds.close()
    # break
print("所有文件处理完成！")

In [ ]:
# 冻土区面积箱型图
import matplotlib.pyplot as plt
import numpy as np

# 原始数据
T_P = np.array([0.498, 0.498, 0.496, 0.495, 0.496, 0.496, 0.495, 0.495, 0.493,
       0.49 , 0.486, 0.483, 0.481, 0.479, 0.478, 0.478, 0.477, 0.475,
       0.475, 0.471, 0.465, 0.462, 0.464, 0.461, 0.458])*100
N_p = np.array([0.502, 0.502, 0.504, 0.505, 0.504, 0.504, 0.505, 0.505, 0.507,
       0.51 , 0.514, 0.517, 0.519, 0.521, 0.522, 0.522, 0.523, 0.525,
       0.525, 0.529, 0.535, 0.538, 0.536, 0.539, 0.542])*100
years = list(range(1997, 2022))

# 创建图形
plt.figure(figsize=(12, 7), dpi=600)
# 绘制折线图
plt.plot(years, T_P, 'o-', color='#0A3C64', linewidth=2, label='T_P')
plt.plot(years, N_p, 's-', color='#2469A2', linewidth=2, label='N_p')

# 五年分组
T_P_groups = [T_P[i:i+5] for i in range(0, 25, 5)]
N_p_groups = [N_p[i:i+5] for i in range(0, 25, 5)]
period_centers = [1999, 2004, 2009, 2014, 2019]  # 五年期的中心年份

# 绘制箱型图  "#2469A2", "#0A3C64"
plt.boxplot(T_P_groups, positions=np.array(period_centers), widths=2, 
           patch_artist=True, showmeans=True,
           boxprops=dict(facecolor='#2469A2', alpha=0.8),
           meanprops=dict(marker='D', markeredgecolor="#2469A2", markerfacecolor='#2469A2'))

plt.boxplot(N_p_groups, positions=np.array(period_centers), widths=2, 
           patch_artist=True, showmeans=True,
           boxprops=dict(facecolor='#0A3C64', alpha=0.8),
           meanprops=dict(marker='D', markeredgecolor="#0A3C64", markerfacecolor='#0A3C64'))
# 在箱型图内部显示均值
# 自定义文本位置 - 您可以调整这些值
text_positions = {
    'T_P': [49.1, 48.7, 47.5, 46.8, 45.55],  # 每个T_P箱型图的文本y坐标
    'N_p': [50.8, 51.2, 52.5, 53.1, 54.4]   # 每个N_p箱型图的文本y坐标
}
for i, (t_group, n_group) in enumerate(zip(T_P_groups, N_p_groups)):
    t_mean = np.mean(t_group)
    n_mean = np.mean(n_group)
    
    # 在箱型图内部显示均值
    plt.text(period_centers[i], text_positions['T_P'][i], f'{t_mean:.1f}', 
             ha='center', va='center', fontsize=20, color='black', fontweight='bold')
    plt.text(period_centers[i], text_positions['N_p'][i], f'{n_mean:.1f}', 
             ha='center', va='center', fontsize=20, color='black', fontweight='bold')
# 设置图形
plt.ylabel('percentage (%)', fontsize=30)
plt.grid(True, alpha=0.3)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.tight_layout()
plt.show()